In [1]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np


DEVICE = torch.device('cpu')
OUTPUT_SIZE = 2048

model = models.resnext50_32x4d(weights=models.ResNeXt50_32X4D_Weights.IMAGENET1K_V2)

extraction_layer = model._modules.get('avgpool')
model.to(DEVICE)
model.eval()

scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

def get_vec(arg, model, extraction_layer):
    image = normalize(to_tensor(scaler(arg))).unsqueeze(0).to(DEVICE)
    result = torch.zeros(1, OUTPUT_SIZE, 1, 1)
    def copy_data(m, i, o):
        result.copy_(o.data)
    hooked = extraction_layer.register_forward_hook(copy_data)
    with torch.no_grad():
        model(image)
    hooked.remove()
    return result

Downloading: "https://download.pytorch.org/models/resnext50_32x4d-1a0047aa.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-1a0047aa.pth
100%|██████████| 95.8M/95.8M [00:00<00:00, 124MB/s]


In [2]:
import arrow
import base64
import pandas as pd
from glob import iglob
from io import BytesIO
from os.path import basename
from PIL import Image

TEST = '/kaggle/input/fruit-and-vegetable-image-recognition/test'
THUMBNAIL_SIZE = (128, 128)
TRAIN = '/kaggle/input/fruit-and-vegetable-image-recognition/train'
VALID = '/kaggle/input/fruit-and-vegetable-image-recognition/validation'


def embed(model, filename: str):
    with Image.open(fp=filename, mode='r') as image:
        return get_vec(arg=image.convert('RGB'), model=model, extraction_layer=extraction_layer).numpy().reshape(OUTPUT_SIZE,)


# https://stackoverflow.com/a/952952
def flatten(arg):
    return [x for xs in arg for x in xs]

def png(filename: str) -> str:
    with Image.open(fp=filename, mode='r') as image:
        buffer = BytesIO()
        # our images are pretty big; let's shrink the hover images to thumbnail size
        image.resize(size=THUMBNAIL_SIZE).convert('RGB').save(buffer, format='png')
        return 'data:image/png;base64,' + base64.b64encode(buffer.getvalue()).decode()

def get_picture_from_glob(arg: str, tag: str,) -> list:
    time_get = arrow.now()
    result = [pd.Series(data=[tag, basename(input_file), embed(model=model, filename=input_file), png(filename=input_file)],
                        index=['tag', 'name', 'value', 'png'])
        for input_file in iglob(pathname=arg) if input_file.lower().endswith('.jpg')]
    print('encoded {} rows of {}  in {}'.format(len(result), tag, arrow.now() - time_get))
    return result

time_start = arrow.now()
test_dict = {basename(folder) : folder + '/*.*' for folder in iglob(TEST + '/*')}
valid_dict = {basename(folder) : folder + '/*.*' for folder in iglob(VALID + '/*')}
train_dict = {basename(folder) : folder + '/*.*' for folder in iglob(TRAIN + '/*')}
test_df = pd.DataFrame(data=flatten(arg=[get_picture_from_glob(arg=value, tag=key) for key, value in test_dict.items()]))
print('{} loaded test data'.format(arrow.now() - time_start))
valid_df = pd.DataFrame(data=flatten(arg=[get_picture_from_glob(arg=value, tag=key) for key, value in valid_dict.items()]))
print('{} loaded valid data'.format(arrow.now() - time_start))
train_df = pd.DataFrame(data=flatten(arg=[get_picture_from_glob(arg=value, tag=key) for key, value in train_dict.items()]))
print('done in {}'.format(arrow.now() - time_start))

encoded 10 rows of capsicum  in 0:00:03.117923
encoded 10 rows of sweetcorn  in 0:00:02.127022
encoded 7 rows of orange  in 0:00:02.484427
encoded 10 rows of tomato  in 0:00:02.497618
encoded 10 rows of turnip  in 0:00:01.788443
encoded 10 rows of ginger  in 0:00:02.726567
encoded 8 rows of raddish  in 0:00:01.390288
encoded 10 rows of pomegranate  in 0:00:02.754273
encoded 10 rows of pineapple  in 0:00:04.676548
encoded 9 rows of jalepeno  in 0:00:02.540864
encoded 10 rows of apple  in 0:00:02.993483
encoded 8 rows of carrot  in 0:00:01.471079
encoded 9 rows of lettuce  in 0:00:02.109611
encoded 9 rows of bell pepper  in 0:00:05.736955
encoded 10 rows of eggplant  in 0:00:03.103948
encoded 10 rows of beetroot  in 0:00:02.568640
encoded 10 rows of kiwi  in 0:00:01.784971
encoded 10 rows of pear  in 0:00:02.148396
encoded 10 rows of cabbage  in 0:00:02.566243
encoded 10 rows of cauliflower  in 0:00:02.694458
encoded 10 rows of paprika  in 0:00:02.142350
encoded 7 rows of lemon  in 0:00:

In [3]:
import arrow
from umap import UMAP

time_start = arrow.now()
umap = UMAP(random_state=2024, verbose=False, n_jobs=1, low_memory=False, n_epochs=201)
train_df[['x', 'y']] = umap.fit_transform(X=train_df['value'].apply(func=pd.Series))
valid_df[['x', 'y']] = umap.transform(X=valid_df['value'].apply(func=pd.Series))
test_df[['x', 'y']] = umap.transform(X=test_df['value'].apply(func=pd.Series))
print('done with UMAP in {}'.format(arrow.now() - time_start))

done with UMAP in 0:00:39.212916
